In [17]:
from typing import Sequence, Optional


class Metric:
    def __init__(self, 
                 hypothesis: str = None,
                 references: Optional[Sequence[Sequence[str]]] = None,
                obj: object = None,
                ):
        self.hypothesis = hypothesis
        self.references = references
        self.obj = obj
        
        # True if the position of hypothesis comes after references
        self.reverse = False
        
    def get_method(self, method_name: str):
        return getattr(self.obj, method_name)
    

    def get_score(self, method_name, corpus_level=False, **kargs):
        if corpus_level:
            self.hypothesis = [self.hypothesis]
            self.references = [self.references]
        
        if self.reverse:
            return self.get_method(method_name)(self.references, self.hypothesis)
        return self.get_method(method_name)(self.hypothesis, self.references, kargs)
    
import nltk
from nltk.translate import *

#from .base import Metric
from typing import Sequence, Optional

NAME = ['bleu', 'gleu', 'nist', 'ribes']
# nltk.translate -> ribes_score -> sentence_ribes

class N(Metric):
    def __init__(self, name: str,
                smooth_method: bool = False
                ):
        super().__init__()
        
        assert name.lower() in NAME, 'Wrong tokenizer name!'
        self.name = name.lower() #ribes
        self.obj = nltk.translate
        self.tokenizer = super().get_method(f'{self.name}_score') #ribes_score
        self.obj = self.tokenizer
        self.method_sentence = f'sentence_{self.name}' #sentence_ribes
        self.method_corpus = f'corpus_{self.name}'
        self.reverse = True
        
        # smoothing_function
        self.smooth_method = smooth_method
        self.smoothing_function = None
        if self.smooth_method:
            self.smoothing_function = self.tokenizer.SmoothingFunction().method1
          
        
    def get_score(self, sys_level: bool = False) -> float:
        if sys_level:
            return super().get_score(method_name=self.method_corpus,
                                    corpus_level=True,
                                    smoothing_function=self.smoothing_function
                                    ) 
        else:
            return super().get_score(method_name=self.method_sentence,
                                 corpus_level=False,
                                    smoothing_function=self.smoothing_function
                                    )
        
 

In [20]:
test = N('bleu')
test.hypothesis = 'hello world'
test.references = ['hello word']

In [21]:
test.get_score()

0.8070557274927981